In [30]:
import pandas as pd
import os
#url_list = [url1]#
import pandas as pd
from bs4 import BeautifulSoup
import requests
def get_dribbleshots(years,ps = False):
    dribbles=['0%20Dribbles','1%20Dribble','2%20Dribbles','3-6%20Dribbles','7%2B%20Dribbles']
    terms = ['0','1','2','3_6','7+']
    folder = '/player_shooting/'
    sfolder=''
    stype = "Regular%20Season"
    if ps == True:
        stype="Playoffs"
        sfolder = "/playoffs"
    dataframe=[]
    for year in years:
        i = 0
        for dribble in dribbles:
            #print(dribble)
            season = str(year)+'-'+str(year+1 - 2000)
            part1 = "https://stats.nba.com/stats/leaguedashplayerptshot?CloseDefDistRange=&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&DribbleRange="
            part2 = "&GameScope=&GameSegment=&GeneralRange=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season="

            part3 = "&SeasonSegment=&SeasonType="+stype+"&ShotClockRange=&ShotDistRange=&StarterBench=&TeamID=0&TouchTimeRange=&VsConference=&VsDivision=&Weight="
            url = part1+dribble+part2+season+part3
            headers = {
                    "Host": "stats.nba.com",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate, br",

                    "Connection": "keep-alive",
                    "Referer": "https://stats.nba.com/"
                }
            json = requests.get(url,headers = headers).json()
            print(json.keys())
            data = json["resultSets"][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            json = requests.get(url,headers = headers).json()
            data = json['resultSets'][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            new_columns = {'FG2A_FREQUENCY':'2FG FREQ%',
             'FG2_PCT':'2FG%',
             'FG2A':'2FGA',
             'FG2M':'2FGM',
             'FG3A_FREQUENCY':'3FG FREQ%',
             'FG3_PCT':'3P%',
             'FG3A':'3PA',
             'FG3M':'3PM',
             'EFG_PCT':'EFG%',
             'FG_PCT':'FG%',
             'FGA_FREQUENCY':'FREQ%',
             'PLAYER_NAME':'PLAYER',
             'PLAYER_LAST_TEAM_ABBREVIATION':'TEAM'}
            df = df.rename(columns = new_columns)
            df = df [['PLAYER', 'TEAM', 'AGE', 'GP', 'G', 'FREQ%', 'FGM', 'FGA', 'FG%',
                   'EFG%', '2FG FREQ%', '2FGM', '2FGA', '2FG%', '3FG FREQ%', '3PM', '3PA',
                   '3P%']]
            for col in df.columns:
                if '%' in col or 'PERC' in col:
                    df[col]*=100
            term = terms[i]
            df['dribbles'] = term
            df['year']=year+1
            dataframe.append(df)

            i+=1
        return pd.concat(dataframe)

def master_dribble(year,ps = False):
    trail='_ps'
    if ps == False:
        trail=''
    old = pd.read_csv('dribbleshot'+trail+'.csv')
    df = get_dribbleshots([year],ps=True)
    year+=1

    old = old[old.year!=year]
    new_master = pd.concat([old,df])
    new_master.to_csv('dribbleshot'+trail+'.csv',index=False)
    return new_master
def get_dribbleshots2(years,ps = False):
    dribbles=['0%20Dribbles','1%20Dribble','2%20Dribbles','3-6%20Dribbles','7%2B%20Dribbles']
    terms = ['0','1','2','3_6','7+']
    folder = '/player_shooting/'
    sfolder=''
    stype = "Regular%20Season"
    if ps == True:
        stype="Playoffs"
        sfolder = "/playoffs"
    dataframe=[]
    for year in years:
        i = 0
        for dribble in dribbles:
            #print(dribble)
            season = str(year)+'-'+str(year+1 - 2000)
            part1 = "https://stats.nba.com/stats/leaguedashplayerptshot?CloseDefDistRange=&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&DribbleRange="
            part2 = "&GameScope=&GameSegment=&GeneralRange=Catch%20and%20Shoot&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season="

            part3 = "&SeasonSegment=&SeasonType="+stype+"&ShotClockRange=&ShotDistRange=&StarterBench=&TeamID=0&TouchTimeRange=&VsConference=&VsDivision=&Weight="
            url = part1+dribble+part2+season+part3
            headers = {
                    "Host": "stats.nba.com",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate, br",

                    "Connection": "keep-alive",
                    "Referer": "https://stats.nba.com/"
                }
            json = requests.get(url,headers = headers).json()
            print(json.keys())
            data = json["resultSets"][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            json = requests.get(url,headers = headers).json()
            data = json['resultSets'][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            new_columns = {'FG2A_FREQUENCY':'2FG FREQ%',
             'FG2_PCT':'2FG%',
             'FG2A':'2FGA',
             'FG2M':'2FGM',
             'FG3A_FREQUENCY':'3FG FREQ%',
             'FG3_PCT':'3P%',
             'FG3A':'3PA',
             'FG3M':'3PM',
             'EFG_PCT':'EFG%',
             'FG_PCT':'FG%',
             'FGA_FREQUENCY':'FREQ%',
             'PLAYER_NAME':'PLAYER',
             'PLAYER_LAST_TEAM_ABBREVIATION':'TEAM'}
            df = df.rename(columns = new_columns)
            df = df [['PLAYER', 'TEAM', 'AGE', 'GP', 'G', 'FREQ%', 'FGM', 'FGA', 'FG%',
                   'EFG%', '2FG FREQ%', '2FGM', '2FGA', '2FG%', '3FG FREQ%', '3PM', '3PA',
                   '3P%']]
            for col in df.columns:
                if '%' in col or 'PERC' in col:
                    df[col]*=100
            term = terms[i]
            df['dribbles'] = term
            df['year']=year+1
            dataframe.append(df)
          
            part1 = "https://stats.nba.com/stats/leaguedashplayerptshot?CloseDefDistRange=&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&DribbleRange="
            part2 = "&GameScope=&GameSegment=&GeneralRange=Pullups&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season="

            part3 = "&SeasonSegment=&SeasonType="+stype+"&ShotClockRange=&ShotDistRange=&StarterBench=&TeamID=0&TouchTimeRange=&VsConference=&VsDivision=&Weight="
            url = part1+dribble+part2+season+part3
            headers = {
                    "Host": "stats.nba.com",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                    "Accept": "application/json, text/plain, */*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate, br",

                    "Connection": "keep-alive",
                    "Referer": "https://stats.nba.com/"
                }
            json = requests.get(url,headers = headers).json()
            print(json.keys())
            data = json["resultSets"][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            json = requests.get(url,headers = headers).json()
            data = json['resultSets'][0]["rowSet"]
            columns = json["resultSets"][0]["headers"]
            df = pd.DataFrame.from_records(data, columns=columns)
            new_columns = {'FG2A_FREQUENCY':'2FG FREQ%',
             'FG2_PCT':'2FG%',
             'FG2A':'2FGA',
             'FG2M':'2FGM',
             'FG3A_FREQUENCY':'3FG FREQ%',
             'FG3_PCT':'3P%',
             'FG3A':'3PA',
             'FG3M':'3PM',
             'EFG_PCT':'EFG%',
             'FG_PCT':'FG%',
             'FGA_FREQUENCY':'FREQ%',
             'PLAYER_NAME':'PLAYER',
             'PLAYER_LAST_TEAM_ABBREVIATION':'TEAM'}
            df = df.rename(columns = new_columns)
            df = df [['PLAYER_ID','PLAYER', 'TEAM', 'AGE', 'GP', 'G', 'FREQ%', 'FGM', 'FGA', 'FG%',
                   'EFG%', '2FG FREQ%', '2FGM', '2FGA', '2FG%', '3FG FREQ%', '3PM', '3PA',
                   '3P%']]
            for col in df.columns:
                if '%' in col or 'PERC' in col:
                    df[col]*=100
            term = terms[i]
            df['dribbles'] = term
            df['year']=year+1
            i+=1
            dataframe.append(df)
        return pd.concat(dataframe)

def master_jump(year,ps = False):
    trail='_ps'
    if ps == False:
        trail=''
    old = pd.read_csv('jumpdribble'+trail+'.csv')
    df = get_dribbleshots2([year],ps=True)
    df2 = df.groupby(['PLAYER_ID','PLAYER', 'TEAM', 'AGE', 'GP', 'G','dribbles','year']).sum()[['FGM', 'FGA','2FGM', '2FGA', '3PM', '3PA',]].reset_index()
    year+=1

    old = old[old.year!=year]
    new_master = pd.concat([old,df2])
    new_master.to_csv('jumpdribble'+trail+'.csv',index=False)
    return new_master
#df = master_dribble(2023,ps=True)
df = master_jump(2023,ps = True)
df

dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])
dict_keys(['resource', 'parameters', 'resultSets'])


/tmp/ipykernel_5102/424779837.py:199: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dataframe)


,PLAYER_ID,PLAYER,dribbles,year,FGM,FGA,2FGM,2FGA,3PM,3PA,GP,TEAM,AGE,G
0,708.0,Kevin Garnett,0,2014,8,17,8,16,0,1,12,NaN,NaN,NaN
1,708.0,Kevin Garnett,1,2014,5,12,5,11,0,1,12,NaN,NaN,NaN
2,708.0,Kevin Garnett,3_6,2014,0,1,0,1,0,0,12,NaN,NaN,NaN
3,951.0,Ray Allen,0,2014,29,77,0,5,29,72,20,NaN,NaN,NaN
4,951.0,Ray Allen,1,2014,3,10,1,5,2,5,20,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,1641722.0,Jordan Hawkins,1,2024,0,1,0,0,0,1,3,NOP,22.0,1.0
383,1641722.0,Jordan Hawkins,2,2024,0,1,0,0,0,1,3,NOP,22.0,1.0
384,1641724.0,Jett Howard,3_6,2024,1,2,0,1,1,1,2,ORL,20.0,2.0
385,1641748.0,Andre Jackson Jr.,3_6,2024,0,1,0,0,0,1,4,MIL,22.0,1.0


In [24]:
df.columns

Index(['PLAYER_ID', 'PLAYER', 'dribbles', 'year', 'FGM', 'FGA', '2FGM', '2FGA',
       '3PM', '3PA', 'GP', 'TEAM', 'AGE', 'G', 'FREQ%', 'FG%', 'EFG%',
       '2FG FREQ%', '2FG%', '3FG FREQ%', '3P%'],
      dtype='object')

,PLAYER_ID,PLAYER,TEAM,AGE,GP,G,dribbles,year,FGM,FGA,2FGM,2FGA,3PM,3PA
0,2544.0,LeBron James,LAL,39.0,5,1,2,2024,0,3,0,2,0,1
1,2544.0,LeBron James,LAL,39.0,5,5,3_6,2024,6,8,5,6,1,2
2,2544.0,LeBron James,LAL,39.0,5,5,7+,2024,3,13,2,6,1,7
3,200768.0,Kyle Lowry,PHI,38.0,5,1,7+,2024,1,1,1,1,0,0
4,200768.0,Kyle Lowry,PHI,38.0,5,2,1,2024,1,2,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,1641722.0,Jordan Hawkins,NOP,22.0,3,1,1,2024,0,1,0,0,0,1
383,1641722.0,Jordan Hawkins,NOP,22.0,3,1,2,2024,0,1,0,0,0,1
384,1641724.0,Jett Howard,ORL,20.0,2,2,3_6,2024,1,2,0,1,1,1
385,1641748.0,Andre Jackson Jr.,MIL,22.0,4,1,3_6,2024,0,1,0,0,0,1
